In [1]:
# hai hai hai, huggingface sega lai la!
from transformers import pipeline

sega = pipeline("text2text-generation",model='beyond/sega-large', device=1)

In [2]:
sketch = "<mask> Conference on Empirical Methods <mask> submission of research papers <mask> Deep Learning <mask>"
generated_text = sega(sketch, num_beams=3, do_sample=True, max_length=200)[0]['generated_text']
generated_text

'The Conference on Empirical Methods welcomes the submission of research papers. Abstracts should be in the form of a paper or presentation. Please submit abstracts to the following email address: eemml.stanford.edu. The conference will be held at Stanford University on April 1618, 2019. The theme of the conference is Deep Learning.'

In [1]:
from transformers import pipeline
# model_path = 'saved_models/bart-large-c4-l_50_200-d_13799838-yake_mask-t_3900800/checkpoint-91425'
model_path = 'saved_models/bart-large-c4-l_50_200-d_13799838-yake_mask-t_3900800/checkpoint-152375'
sega = pipeline("text2text-generation",model=model_path, device=0)

In [37]:

sega('Bad news: <mask> the European Union <mask> month by EU <mask> Farm Commissioner Franz <mask>', num_beams=3, do_sample=True, max_length=200)

[{'generated_text': "Bad news: the European Union will not be able to increase the price of corn until next month by EU's Farm Commissioner Franz means. That's according to a report from Reuters. The report says the EU will have to raise the price by 1.5 percent next year. Corn prices have been rising steadily for the past few years, and are now at their highest level since 2007."}]

In [33]:
kws = ['Mickle Jordan', 'in China']
sketch = 'business: ' + '<mask> '+' <mask> '.join(kws)+' <mask>'
print(sketch)
sega(sketch, num_beams=3, do_sample=True, max_length=200)

business: <mask> Mickle Jordan <mask> in China <mask>


[{'generated_text': 'business: "Mickle Jordan has done a great job for us in China. He\'s been very helpful to us. We look forward to working with him in the future. I think he\'s going to be a great asset to us in the near future."'}]

In [4]:
sega("<mask> nothing compelling <mask> no real cats <mask> prophecies of Armageddon <mask> of an Australian <mask>", max_length=200)


[{'generated_text': "There's nothing compelling here. There are no real cats. There's no prophecies of Armageddon. It's just the story of an Australian couple who are trying to make it as a family in a small town in the middle of nowhere."}]

In [6]:
def sega_gen(kws,n):
    M = ' <mask> '
    t = M.join(kws)
    res = sega(t, num_beams=3, do_sample=True, num_return_sequences=n, max_length=100)
    res = [each['generated_text'] for each in res]
    return res

In [7]:
kws = ['what a nice day', 'I have a plan', 'sun beach', 'drink and food', 'surfing', 'swimming']
sega_gen(kws, 3)

['what a nice day! I have a plan for the sun beach. I like to drink and food, surfing, swimming, etc. I would like to go on a bike ride and have a good time. I want to go to the beach with my friends and family.',
 'what a nice day! I have a plan for the sun beach, drink and food, surfing, swimming, etc. I am looking forward to it! I am also looking for a place for my friends and family to go for a swim. I want to have fun and enjoy it.',
 'what a nice day. I have a plan. sun beach with a drink and food. surfing, swimming and more. I am looking forward to the day.']

In [9]:
kws = ['I like machine learning', 'join a conference', 'doing research', 'natural language processing']
sega_gen(kws, 3)

['I like machine learning. I like to join a conference, doing research, and learning about natural language processing. I have a passion for machine learning, but I am also interested in learning about the future of AI.',
 'I like machine learning, so when I join a conference, I think about doing research on natural language processing. I like the idea of using AI to solve problems. I think that it makes a lot of sense to do research on things like this.',
 'I like machine learning. I would like to join a conference where I would be doing research on natural language processing. I also like to learn how to use AI to improve the performance of computers.']

In [12]:
kws = ['Chess world <mask> Magnus Carlsen outright <mask> Hans Niemann of cheating.']
sega_gen(kws, 3)

['Chess world No. 1 Magnus Carlsen outright has been accused by Hans Niemann of cheating.',
 'Chess world No. 1 Magnus Carlsen outright is accused by Hans Niemann of cheating..',
 'Chess world No. 1 Magnus Carlsen outright has been accused by Hans Niemann of cheating.']

In [1]:
from transformers import pipeline

class K2T_Generator:
    def __init__(self,model, device):
        self.generator = pipeline("text2text-generation", model=model, device=device)
    def k2t(self, inputs, beam=1, sample=False, max_length=200):
        """
        inputs: text or a list of text
        """
        res = self.generator(inputs, num_beams=beam, do_sample=sample, num_return_sequences=1, max_length=max_length)
        return [item['generated_text'] for item in res]

In [2]:
## 10000 documents for pre-training
model = f'saved_models/bart-large-c4-l_50_200-d_1000000-yake_mask-t_281571/checkpoint-6600'
# model = f'facebook/bart-large'
generator = K2T_Generator(model, device=1)

In [ ]:
generator.k2t('what the fuck is going hell on?')

In [ ]:
import time
t = time.time()
ss = ['nice weekend movie fun',
      'Shanghai food water help citizens virus',
      'England ship China ocean Paris cake people',
      'joint great food great drinks greater staff',
      'Wuhan hot-dry noodel delicious breakfirst China street Hubuxiang tour nice place',]*2
for s in ss:
    s = s.replace(' ',' <mask> ')
    s = '<mask> '+s+' <mask>'
    print(generator.k2t(s))
print(time.time()-t,'s')

In [ ]:
from torch.utils.data import Dataset
class MyDataset(Dataset):
    def __init__(self, ss):
        self.ss = ss
        self.ss = ['<mask> '+s.replace(' ',' <mask> ')+' <mask>' for s in ss]
    def __len__(self):
        return len(self.ss)
    def __getitem__(self, i):
        return self.ss[i]
data = MyDataset(ss)
data.__getitem__(-1)

In [ ]:
t = time.time()
for out in generator.generator(data,num_beams=5, do_sample=True, num_return_sequences=1, max_length=200,batch_size=10):
    print(out)
print(time.time()-t,'s')

In [ ]:
import pandas as pd
dataset_name = 'imdb_50'
data = pd.read_csv(f'clf_data/{dataset_name}/train.csv')
data = data.dropna()
data = data[data.content != ''] # 处理空值问题
contents = list(data.content)
labels = list(data.label)

In [ ]:

# 抽出最大为3的ngram

def mask_unimportant_parts(text, max_ngram=3, topk=20):
    ke_yake = yake.KeywordExtractor(n=max_ngram,top=topk)
    kws_paris = ke_yake.extract_keywords(text) 
    kws =  [pair[0] for pair in kws_paris]

    words_idxs = []
    all_ids = []
    for w in kws: # 找出每个词的位置
        for m in list(re.finditer(w,text)): 
            all_ids += list(range(m.start(),m.end()))
    all_ids = sorted(list(set(all_ids)))
    # 给不连续的部分中间补上mask token
    masked_text = []
    for i,id in enumerate(all_ids):
        if i == 0 and id != 0: # 开头补mask
            masked_text.append('<mask> ')
        if id - all_ids[i-1] > 1: # 说明中间有东西
            masked_text.append(' <mask> ')
        masked_text.append(text[id])
        if i == len(all_ids)-1 and id != len(text)-1: # 最后补mask
            masked_text.append(' <mask>')
    masked_text = ''.join(masked_text)
    return masked_text

mask_unimportant_parts(text)

In [ ]:
def show(i):
    print(labels[i], contents[i])
for i in range(10):
    print('>>>',i)
    show(i)

In [ ]:
m = mask_unimportant_parts(contents[4],topk=20)
m

In [ ]:
import random
elements = m.split('<mask>')
shuffle_elements = random.shuffle(elements)

In [ ]:
generator.k2t(m)